Bayan Nezamabad - 20251971, Euan Bourke - 21332142

In [16]:
# Imports
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers
import matplotlib.pyplot as plt

In [17]:
# Hyper Params
EPOCHS = 20
BATCH_SIZE = 128
VERBOSE = 1
OPTIMIZER = optimizers.Adam()
VALIDATION_SPLIT=0.90

IMG_ROW, IMG_COL = 224, 224 # Image dimensions
INPUT_SHAPE = (IMG_ROW, IMG_COL, 3) # 3 as RGB
NB_CLASSES = 100 # 100 different classifications of species

In [18]:
class Dataset:
    def __init__(self):
        return
        

In [19]:
class BENet: # Our implementation of AlexNet
    def __init__(self):
        self.model = tf.keras.models.Sequential()
        self.build_layers()

    def build_layers(self):

        # Convolutional Layers
        self.model.add(layers.Conv2D(96, (11, 11), strides=(4, 4), activation='relu', input_shape=INPUT_SHAPE))
        self.model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=2))
        self.model.add(layers.BatchNormalization())

        self.model.add(layers.Conv2D(256, (5, 5), padding='same', activation='relu'))
        self.model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=2))
        self.model.add(layers.BatchNormalization())

        self.model.add(layers.Conv2D(384, (3, 3), padding='same', activation='relu'))
        self.model.add(layers.Conv2D(384, (3, 3), padding='same', activation='relu'))
        self.model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu'))

        self.model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=2))

        self.model.add(layers.Flatten()) # Flatten Layers

        # Full Connected Layers
        self.model.add(layers.Dropout(rate=0.5))
        self.model.add(layers.Dense(4096, activation='relu'))

        self.model.add(layers.Dropout(rate=0.5))
        self.model.add(layers.Dense(4096, activation='relu'))
        
        self.model.add(layers.Dense(NB_CLASSES, activation='softmax'))
        
        return model




model = BENet()

        

/Users/euanbourke/Documents/cs4287-assignment1/.venv/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Refs: \
AlexNet Structure: https://www.kaggle.com/code/blurredmachine/alexnet-architecture-a-complete-guide \
Keras.layer docs: https://www.tensorflow.org/api_docs/python/tf/keras/layers/
